In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/Users/austin-pan/Documents/lynx'

In [2]:
import pandas as pd
import numpy as np
import lynx as lx

dataset_path = "/Users/austin-pan/Downloads/lynx/datasets/bookcrossing/BX-CSV-Dump"

In [3]:
books = pd.read_csv(
    f"{dataset_path}/BX-Books.csv",
    # names=["ISBN", "Book-Title", "Book-Author", "Year-Of-Publication", "Publisher", "Image-URL-S", "Image-URL-M", "Image-URL-L"],
    usecols=["ISBN", "Book-Author"],
    sep=";",
    # engine="python",
    encoding="cp1252",
    on_bad_lines='skip',
    escapechar='\\',
    nrows=1000
)
books_table = lx.Table(books, "books")

In [4]:
users = pd.read_csv(
    f"{dataset_path}/BX-Users.csv",
    # names=["User-ID", "Location", "Age"],
    usecols=["User-ID", "Age"],
    sep=";",
    # engine="python",
    encoding="cp1252",
    nrows=1000
)
users_table = lx.Table(users, "users")

In [5]:
ratings = pd.read_csv(
    f"{dataset_path}/BX-Book-Ratings.csv",
    # names=["User-ID", "ISBN", "Book-Rating"],
    usecols=["User-ID", "ISBN", "Book-Rating"],
    sep=";",
    # engine="python",
    encoding="cp1252",
    nrows=10000
)
ratings_table = lx.Table(ratings, "ratings")

In [11]:
merged_table = (ratings_table
    # .merge(users_table, left_on="User-ID", right_on="User-ID")
    # .merge(books_table, left_on="ISBN", right_on="ISBN")
    .model_feature_interactions("User-ID", "ISBN", drop=False)
    .onehot("User-ID")
    .onehot("ISBN")
)
merged_table.to_dataframe().head()

,Book-Rating,0,1,2,3,4,5,6,7,8,...,9330,9331,9332,9333,9334,9335,9336,9337,9338,9339
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
from sklearn.model_selection import train_test_split

def rmse(predictions, targets) -> float:
    return np.sqrt(np.mean((predictions - targets) ** 2))

seed = 0

y = merged_table.pop("Book-Rating")
X_train, X_test, y_train, y_test = train_test_split(
    merged_table, y,
    train_size=0.8,
    test_size=0.2,
    random_state=seed
)

In [13]:
from lynx.libfm import mcmc

mcmc_r = mcmc.FMRegression(seed=seed)
pred = mcmc_r.fit_predict(X_train, y_train, X_test, verbose=True)
mcmc_r.flush()

rmse(pred, y_test)

Converting /tmp/libfm_mcmc_r_1713101532660775/train
Transposing /tmp/libfm_mcmc_r_1713101532660775/train
Converting /tmp/libfm_mcmc_r_1713101532660775/test
Transposing /tmp/libfm_mcmc_r_1713101532660775/test
/Users/austin-pan/Documents/libfm/bin/libFM -method mcmc -task r -train train -test test -dim 1,1,8 -iter 100 -init_stdev 0.1 -out /tmp/libfm_mcmc_r_1713101532660775/predictions.txt -seed 0
----------------------------------------------------------------------------
libFM
  Version: 1.4.4
  Author:  Steffen Rendle, srendle@libfm.org
  WWW:     http://www.libfm.org/
This program comes with ABSOLUTELY NO WARRANTY; for details see license.txt.
This is free software, and you are welcome to redistribute it under certain
conditions; for details see license.txt.
----------------------------------------------------------------------------
Loading train...
has x = 0
has xt = 1
data transpose... num entries in cache=778422	num rows in cache=19621
num_cases=8000	num_values=778422	num_features

2.8328969569784324